In [1]:
from tweet_sentiment_service.model import SentimentModel
from tweet_sentiment_service.inference import SentimentExtractor, SentimentID
from tokenizers import ByteLevelBPETokenizer
import pandas as pd
import numpy as np
from fastapi.testclient import TestClient
from tweet_sentiment_service.app import app

#### Test Model Class only

In [2]:
MAX_LEN = 96
PATH = './config/'

tokenizer = ByteLevelBPETokenizer(
    vocab=PATH+'vocab-roberta-base.json', 
    merges=PATH+'merges-roberta-base.txt', 
    lowercase=True,
    add_prefix_space=True
)
sentiment_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974}

In [ ]:
# Data preprocessing
test = pd.read_csv('./training/test.csv').fillna('')
tweet = test.loc[2, 'text'] #'Last session of the day  http://twitpic.com/67ezh'
sentiment = 'negative'
ct = 1
input_ids = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids = np.zeros((ct,MAX_LEN),dtype='int32')

# INPUT_IDS
text1 = " "+" ".join(tweet.split())
enc = tokenizer.encode(text1)
s_tok = sentiment_id[sentiment]           
input_ids[0,:len(enc.ids)+5] = [0] + enc.ids + [2,2] + [s_tok] + [2]
attention_mask[0,:len(enc.ids)+5] = 1

# Model Setup
model = SentimentModel(weights_path="./weights_final.h5", config_path=PATH)

In [ ]:
DISPLAY = 1
predictions = model.predict(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids
        )

prediction_start = np.zeros((1,MAX_LEN))
prediction_end = np.zeros((1,MAX_LEN))

prediction_start += predictions[0]
prediction_end += predictions[1]

start_idx = np.argmax(prediction_start)
end_idx = np.argmax(prediction_end)

if start_idx > end_idx:
    selected_text = tweet

text = " "+" ".join(tweet.split())
encoded_text = tokenizer.encode(text)
selected_text = tokenizer.decode(encoded_text.ids[start_idx-1:end_idx])

print(selected_text)

#### Test Inference Module

In [ ]:
PATH = './config/'
# Input
test = pd.read_csv('./training/test.csv').fillna('')
tweet = test.loc[2, 'text'] #'Last session of the day  http://twitpic.com/67ezh'
sentiment = 'negative'

# Inference
sentiment_extractor = SentimentExtractor(weights_path="./weights_final.h5", config_path=PATH)

# Extract Sentiment from Tweet
ans = sentiment_extractor.extract_sentiment(tweet=tweet, sentiment=sentiment)
print(ans)

#### Test API

In [2]:
client = TestClient(app=app)

In [13]:
from tweet_sentiment_service.app import SentimentRequest
data = {
    "tweet": "Recession hit Veronique Branquinho, she has to quit her company, such a shame!",
    "sentiment": "negative",
}
response = client.post(url="/sentiment", json=data)

2024-11-11 22:02:32.522893: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at ./config/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


1/1 [==============================] - 5s 5s/step


In [14]:
response.json()

{'status': 'success',
 'selected_text': ' recession hit veronique branquinho, she has to quit her'}

In [7]:
params = {
    "tweet": "I went on a walk with my dog yesterday, and saw a homeless guy. So sad!",
    "sentiment": "negative",
}
response = client.get(url="/sentiment", params=params)

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at ./config/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


1/1 [==============================] - 6s 6s/step


In [9]:
response.json()

{'selected_text': 'I went on a walk with my dog yesterday, and saw a homeless guy. So sad!'}

In [ ]:
# pip install --upgrade anyio httpx starlette fastapi